In [1]:
import pandas as pd

# Read the datasets
train_up = pd.read_csv(r'C:\Users\mvk\Documents\DATA_school\thesis\Subset\top_price_changers_subset\train_up.csv')
top_price_changers = pd.read_csv(r'C:\Users\mvk\Documents\DATA_school\thesis\Subset\top_price_changers_subset.csv')

# Ensure date column is in datetime format
train_up['date'] = pd.to_datetime(train_up['date'])
top_price_changers['date'] = pd.to_datetime(top_price_changers['date'])

# Merge the datasets based on listing_id and date
train_up2 = train_up.merge(
    top_price_changers[['listing_id', 'date', 'price']], 
    on=['listing_id', 'date'], 
    how='left', 
    suffixes=('_new', '_original')
)

# If price_original is not found, keep the original price
train_up2['price'] = train_up2['price_original'].fillna(train_up2['price_new'])

# Drop the temporary columns
train_up2 = train_up2.drop(columns=['price_new', 'price_original'])

# Save the new dataset
train_up2.to_csv('train_up2.csv', index=False)

print("Matching complete. New dataset saved as train_up2.csv")
print("\nDataset info:")
print(train_up2.info())

# Check how many prices were matched
matched_prices = train_up2['price'].notnull().sum()
total_rows = len(train_up2)
print(f"\nTotal rows: {total_rows}")
print(f"Rows with matched prices: {matched_prices}")
print(f"Percentage of matched prices: {matched_prices/total_rows*100:.2f}%")

Matching complete. New dataset saved as train_up2.csv

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1404469 entries, 0 to 1404468
Data columns (total 46 columns):
 #   Column                          Non-Null Count    Dtype         
---  ------                          --------------    -----         
 0   listing_id                      1404469 non-null  int64         
 1   date                            1404469 non-null  datetime64[ns]
 2   latitude                        1404469 non-null  float64       
 3   longitude                       1404469 non-null  float64       
 4   accommodates                    1404469 non-null  float64       
 5   bedrooms                        1404469 non-null  float64       
 6   has_kitchen                     1404469 non-null  int64         
 7   has_wifi                        1404469 non-null  int64         
 8   has_smoke_alarm                 1404469 non-null  int64         
 9   has_essentials                  1404469 non

In [6]:
import polars as pl

def process_transformed_df(transformed_df):
    # Convert date column to date type 
    transformed_df = transformed_df.with_columns(
        pl.col("date").str.to_date()
    )
    
    # Group processed listings
    processed_listings = []
    
    for listing_id in transformed_df["listing_id"].unique():
        # Filter for current listing
        listing_df = transformed_df.filter(pl.col("listing_id") == listing_id).sort("date")
        
        # Create lag and rolling features
        processed_listing = listing_df.with_columns([
            # Add lag features
            pl.col("price").shift(7).alias("price_lag_7d"),
            pl.col("price").shift(14).alias("price_lag_14d"),
            pl.col("price").shift(30).alias("price_lag_30d"),
            
            # 7-day windows
            pl.col("price").shift(7).rolling_mean(
                window_size=7, 
                min_periods=1,
                center=False
            ).alias("rolling_mean_7d"),
            pl.col("price").shift(7).rolling_max(
                window_size=7, 
                min_periods=1,
                center=False
            ).alias("rolling_max_7d"),
            pl.col("price").shift(7).rolling_min(
                window_size=7, 
                min_periods=1,
                center=False
            ).alias("rolling_min_7d"),
            
            # 14-day windows
            pl.col("price").shift(7).rolling_mean(
                window_size=14, 
                min_periods=1,
                center=False
            ).alias("rolling_mean_14d"),
            pl.col("price").shift(7).rolling_max(
                window_size=14, 
                min_periods=1,
                center=False
            ).alias("rolling_max_14d"),
            pl.col("price").shift(7).rolling_min(
                window_size=14, 
                min_periods=1,
                center=False
            ).alias("rolling_min_14d"),
            
            # 30-day windows
            pl.col("price").shift(7).rolling_mean(
                window_size=30, 
                min_periods=1,
                center=False
            ).alias("rolling_mean_30d"),
            pl.col("price").shift(7).rolling_max(
                window_size=30, 
                min_periods=1,
                center=False
            ).alias("rolling_max_30d"),
            pl.col("price").shift(7).rolling_min(
                window_size=30, 
                min_periods=1,
                center=False
            ).alias("rolling_min_30d")
        ])
        
        # List of columns to fill
        columns_to_fill = [
            "price_lag_7d", "price_lag_14d", "price_lag_30d",
            "rolling_mean_7d", "rolling_max_7d", "rolling_min_7d",
            "rolling_mean_14d", "rolling_max_14d", "rolling_min_14d",
            "rolling_mean_30d", "rolling_max_30d", "rolling_min_30d"
        ]
        
        # Forward fill these columns
        processed_listing = processed_listing.with_columns(
            [pl.col(col).forward_fill().alias(col) for col in columns_to_fill]
        )
        
        # Backward fill for the earliest dates (fill with first available non-null value)
        processed_listing = processed_listing.with_columns(
            [pl.col(col).backward_fill().alias(col) for col in columns_to_fill]
        )
        
        processed_listings.append(processed_listing)
    
    # Combine all processed listings
    final_df = pl.concat(processed_listings)
    
    return final_df

# Read the input CSV
transformed_df = pl.read_csv('train_up2.csv')

# Process the dataframe
result = process_transformed_df(transformed_df)

# Save the processed dataframe
result.write_csv('train_up3.csv')

# Print some information about the processed dataframe
print("Processed dataframe information:")
print(result.glimpse())

# Check the number of rows and columns
print(f"\nNumber of rows: {result.height}")
print(f"Number of columns: {result.width}")

# Check for null values in key columns
null_counts = result.null_count()
print("\nNull value counts:")
print(null_counts)

# Optional: Print the first few rows of a specific listing to verify filling
print("\nSample rows for first listing:")
first_listing = result.filter(pl.col("listing_id") == result["listing_id"].unique()[0])
print(first_listing.head())

C:\Users\mvk\AppData\Local\Temp\ipykernel_4672\40785815.py:24: DeprecationWarning: The argument `min_periods` for `Expr.rolling_mean` is deprecated. It has been renamed to `min_samples`.
  pl.col("price").shift(7).rolling_mean(
C:\Users\mvk\AppData\Local\Temp\ipykernel_4672\40785815.py:29: DeprecationWarning: The argument `min_periods` for `Expr.rolling_max` is deprecated. It has been renamed to `min_samples`.
  pl.col("price").shift(7).rolling_max(
C:\Users\mvk\AppData\Local\Temp\ipykernel_4672\40785815.py:34: DeprecationWarning: The argument `min_periods` for `Expr.rolling_min` is deprecated. It has been renamed to `min_samples`.
  pl.col("price").shift(7).rolling_min(
C:\Users\mvk\AppData\Local\Temp\ipykernel_4672\40785815.py:41: DeprecationWarning: The argument `min_periods` for `Expr.rolling_mean` is deprecated. It has been renamed to `min_samples`.
  pl.col("price").shift(7).rolling_mean(
C:\Users\mvk\AppData\Local\Temp\ipykernel_4672\40785815.py:46: DeprecationWarning: The argum

Processed dataframe information:
Rows: 1404469
Columns: 46
$ listing_id                      <i64> 5396, 5396, 5396, 5396, 5396, 5396, 5396, 5396, 5396, 5396
$ date                           <date> 2023-08-07, 2023-08-08, 2023-08-09, 2023-08-10, 2023-08-11, 2023-08-12, 2023-08-13, 2023-08-14, 2023-08-15, 2023-08-16
$ latitude                        <f64> 48.85247, 48.85247, 48.85247, 48.85247, 48.85247, 48.85247, 48.85247, 48.85247, 48.85247, 48.85247
$ longitude                       <f64> 2.35835, 2.35835, 2.35835, 2.35835, 2.35835, 2.35835, 2.35835, 2.35835, 2.35835, 2.35835
$ accommodates                    <f64> -0.8598353633570076, -0.8598353633570076, -0.8598353633570076, -0.8598353633570076, -0.8598353633570076, -0.8598353633570076, -0.8598353633570076, -0.8598353633570076, -0.8598353633570076, -0.8598353633570076
$ bedrooms                        <f64> -1.5499302366474703, -1.5499302366474703, -1.5499302366474703, -1.5499302366474703, -1.5499302366474703, -1.5499302366474703, 